In [1]:
from langchain import LLMChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.mapreduce import MapReduceChain
from langchain.prompts import PromptTemplate
from langchain.llms.base import LLM
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Qdrant 

from transformers import AutoTokenizer, AutoModel, AutoConfig
from typing import Any, Dict, List, Mapping, Optional, Tuple, Union
from flask import Flask
from flask import render_template
from flask import request
import torch
import sys
import qdrant_client

In [16]:
### 编写GLM类
class GLM(LLM):
    max_token: int = 2048
    temperature: float = 0.8
    top_p = 0.9
    tokenizer: object = None
    model: object = None
    history_len: int = 1024
    
    def __init__(self):
        super().__init__()
        
    @property
    def _llm_type(self) -> str:
        return "GLM"
            
    def load_model(self, llm_device="gpu",model_name_or_path=None):
        model_config = AutoConfig.from_pretrained(model_name_or_path, trust_remote_code=True)
        self.tokenizer = AutoTokenizer.from_pretrained(model_name_or_path,trust_remote_code=True)
        self.model = AutoModel.from_pretrained(model_name_or_path, config=model_config, trust_remote_code=True).half().cuda()

    def _call(self,prompt:str,history:List[str] = [],stop: Optional[List[str]] = None):
        '''
        重写_call方法：加载自己的模型，并限制只输出结果
        （chatglm原输出不是直接str， langchain中要求模型返回必须是str的结果：
        """LLM wrapper should take in a prompt and return a string."""）
        '''
        
        response,_ = self.model.chat(
                    self.tokenizer,prompt,
                    history=history[-self.history_len:] if self.history_len > 0 else [],
                    max_length=self.max_token,temperature=self.temperature,
                    top_p=self.top_p)
        return response

In [3]:
# 实例化LLM
modelpath = "/root/autodl-tmp/chatglm2"
sys.path.append(modelpath)
llm = GLM()
llm.load_model(model_name_or_path = modelpath)

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [4]:
#连接向量数据库
client = qdrant_client.QdrantClient(
    path="./tmp/local_qdrant", prefer_grpc=True
)

In [5]:
## 加载向量化模型
embeddings = HuggingFaceEmbeddings(model_name='/root/ChatBot/QA/text2vec_cmed')

No sentence-transformers model found with name /root/ChatBot/QA/text2vec_cmed. Creating a new one with MEAN pooling.


In [6]:
qdrant = Qdrant(
    client=client, collection_name="my_documents", 
    embedding_function=embeddings.embed_query
)
qdrant.embeddings = embeddings # 需要再设置一下embeddings

/root/miniconda3/lib/python3.8/site-packages/langchain/vectorstores/qdrant.py:102: UserWarning: Using `embedding_function` is deprecated. Pass `Embeddings` instance to `embeddings` instead.
  warnings.warn(
/root/miniconda3/lib/python3.8/site-packages/langchain/vectorstores/qdrant.py:108: UserWarning: `embeddings` should be an instance of `Embeddings`.Using `embeddings` as `embedding_function` which is deprecated
  warnings.warn(


In [10]:
retriever = qdrant.as_retriever()  

In [ ]:
qa = RetrievalQA.from_chain_type( llm=llm, chain_type="stuff",retriever=retriever,     
                                 return_source_documents=True)

In [13]:
text = '项目实施过程中，有没有可能被终止项目'

In [9]:
qdrant.similarity_search(text)

[Document(page_content='科学研究院可视情况撤销、 终止项目， 停拨、 收回剩余经费：  \n（一）存在违法违纪、违反师德、学术不端等行为 。', metadata={'source': 'data/附件2 中山大学关于印发《中山大学高校基本科研业务费管理实施细则》通知.pdf', 'page': 3}),
 Document(page_content='计划项目研究方案变更需经二级单位临床研究中心及伦理\n委员会审核通过后报科学研究院审批。  \n各类项目实施过程中，涉及团队核心成员变更、项目\n延期、项目终止的，需经二级单位审核通 过后报科学研究\n院审批。  \n第十二条  项目实施中， 发生下列情况之一， 一经核实，', metadata={'source': 'data/附件2 中山大学关于印发《中山大学高校基本科研业务费管理实施细则》通知.pdf', 'page': 3}),
 Document(page_content='（二）项目负责人在项目结题前离职 。 \n（三）项目负责人出国半年以上未向科学研究院报备\n或出国逾期不归导致研究工作无法开展 。 \n（四）未通过年度评估 。 \n（五）未通过结题验收 。 \n（六）因其他不可抗力致使研究工作不能正常进行。  \n第十三条  项目负责人应在项目完成时间前 1个月内提', metadata={'source': 'data/附件2 中山大学关于印发《中山大学高校基本科研业务费管理实施细则》通知.pdf', 'page': 4}),
 Document(page_content='（四）平台专项按上级主管部门批复与规定启动。受资\n助单位可根据上级主管部门和学校的规定自行制定使用方\n案，报科学研究院备案。  \n第八条  项目审批立项前， 有下列情形之一者，一经核\n实，科学研究院将不予受理申报或不予资助：  \n（一）项目负责人有违反师德、学术不端行为和近 3年', metadata={'source': 'data/附件2 中山大学关于印发《中山大学高校基本科研业务费管理实施细则》通知.pdf', 'page': 2})]

In [10]:
result = qa({"query": text})  

In [11]:
print(result['result'])

根据给出的 context，项目实施过程中可能被终止项目。具体来说，如果项目存在违法违纪、违反师德、学术不端等行为，或项目负责人在项目结题前离职，或项目负责人出国半年以上未向科学研究院报备或出国逾期不归导致研究工作无法开展，或未通过年度评估，或未通过结题验收，或因其他不可抗力致使研究工作不能正常进行，这些情况一经核实，项目将会被终止。


In [7]:
from langchain.memory import ConversationBufferMemory

In [8]:
template = """
使用下面的上下文(由< ctx > < / ctx >)分隔开的,聊天记录(分隔< hs > < / hs >)来回答这个问题:
------
<ctx>
{context}
</ctx>
------
<hs>
{history}
</hs>
------
{question}
Answer:
"""
prompt = PromptTemplate(
    input_variables=["history", "context", "question"],
    template=template,
)

In [11]:
qa = RetrievalQA.from_chain_type( llm=llm, chain_type="stuff",retriever=retriever,     
                                 return_source_documents=True,
                                chain_type_kwargs={
                                    "verbose": True,
                                    "prompt": prompt,
                                    "memory": ConversationBufferMemory(
                                        memory_key="history",
                                        input_key="question"),
                                })

In [14]:
result = qa({"query": text})  



> Entering new  chain...


> Entering new  chain...
Prompt after formatting:

使用下面的上下文(由< ctx > < / ctx >)分隔开的,聊天记录(分隔< hs > < / hs >)来回答这个问题:
------
<ctx>
科学研究院可视情况撤销、 终止项目， 停拨、 收回剩余经费：  
（一）存在违法违纪、违反师德、学术不端等行为 。

计划项目研究方案变更需经二级单位临床研究中心及伦理
委员会审核通过后报科学研究院审批。  
各类项目实施过程中，涉及团队核心成员变更、项目
延期、项目终止的，需经二级单位审核通 过后报科学研究
院审批。  
第十二条  项目实施中， 发生下列情况之一， 一经核实，

（二）项目负责人在项目结题前离职 。 
（三）项目负责人出国半年以上未向科学研究院报备
或出国逾期不归导致研究工作无法开展 。 
（四）未通过年度评估 。 
（五）未通过结题验收 。 
（六）因其他不可抗力致使研究工作不能正常进行。  
第十三条  项目负责人应在项目完成时间前 1个月内提

（四）平台专项按上级主管部门批复与规定启动。受资
助单位可根据上级主管部门和学校的规定自行制定使用方
案，报科学研究院备案。  
第八条  项目审批立项前， 有下列情形之一者，一经核
实，科学研究院将不予受理申报或不予资助：  
（一）项目负责人有违反师德、学术不端行为和近 3年
</ctx>
------
<hs>

</hs>
------
项目实施过程中，有没有可能被终止项目
Answer:



╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_23311/1593577705.py:1 in <cell line: 1>                                           │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_23311/1593577705.py'                        │
│                                                                                                  │
│ /root/miniconda3/lib/python3.8/site-packages/langchain/chains/base.py:166 in __call__            │
│                                                                                                  │
│   163 │   │   │   )                                                                              │
│   164 │   │   except (KeyboardInterrupt, Exception) as e:                                        │
│   165 │   │   │   run_manager.on_chain_error(e)                                                  │
│ ❱ 166 │   │   │   raise e                                                                        │
│   167 │   │   run_manager.on_chain_end(outputs)                                                  │
│   168 │   │   final_outputs: Dict[str, Any] = self.prep_outputs(                                 │
│   169 │   │   │   inputs, outputs, return_only_outputs                                           │
│                                                                                                  │
│ /root/miniconda3/lib/python3.8/site-packages/langchain/chains/base.py:160 in __call__            │
│                                                                                                  │
│   157 │   │   )                                                                                  │
│   158 │   │   try:                                                                               │
│   159 │   │   │   outputs = (                                                                    │
│ ❱ 160 │   │   │   │   self._call(inputs, run_manager=run_manager)                                │
│   161 │   │   │   │   if new_arg_supported                                                       │
│   162 │   │   │   │   else self._call(inputs)                                                    │
│   163 │   │   │   )                                                                              │
│                                                                                                  │
│ /root/miniconda3/lib/python3.8/site-packages/langchain/chains/retrieval_qa/base.py:131 in _call  │
│                                                                                                  │
│   128 │   │   │   docs = self._get_docs(question, run_manager=_run_manager)                      │
│   129 │   │   else:                                                                              │
│   130 │   │   │   docs = self._get_docs(question)  # type: ignore[call-arg]                      │
│ ❱ 131 │   │   answer = self.combine_documents_chain.run(                                         │
│   132 │   │   │   input_documents=docs, question=question, callbacks=_run_manager.get_child()    │
│   133 │   │   )                                                                                  │
│   134                                                                                            │
│                                                                                                  │
│ /root/miniconda3/lib/python3.8/site-packages/langchain/chains/base.py:293 in run                 │
│                                                                                                  │
│   290 │   │   │   return self(args[0], callbacks=callbacks, tags=tags)[_output_key]              │
│   291 │   │                                                                                      │
│   292 │   │   if kwargs and not args:                      

In [15]:
print(result)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_23311/2443047140.py:1 in <cell line: 1>                                           │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_23311/2443047140.py'                        │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'result' is not defined

In [18]:
result = qa({"query": '项目被终止有什么后果'})  



> Entering new  chain...


> Entering new  chain...
Prompt after formatting:

使用下面的上下文(由< ctx > < / ctx >)分隔开的,聊天记录(分隔< hs > < / hs >)来回答这个问题:
------
<ctx>
科学研究院可视情况撤销、 终止项目， 停拨、 收回剩余经费：  
（一）存在违法违纪、违反师德、学术不端等行为 。

（二）项目负责人在项目结题前离职 。 
（三）项目负责人出国半年以上未向科学研究院报备
或出国逾期不归导致研究工作无法开展 。 
（四）未通过年度评估 。 
（五）未通过结题验收 。 
（六）因其他不可抗力致使研究工作不能正常进行。  
第十三条  项目负责人应在项目完成时间前 1个月内提

计划项目研究方案变更需经二级单位临床研究中心及伦理
委员会审核通过后报科学研究院审批。  
各类项目实施过程中，涉及团队核心成员变更、项目
延期、项目终止的，需经二级单位审核通 过后报科学研究
院审批。  
第十二条  项目实施中， 发生下列情况之一， 一经核实，

类资金支持的在研科研项目 。 
（三）所申报研究内容已获得其他各类科研计划资助 。 
（四）存在无正当理由未按期完成或被终止、撤销所承
担的科研项目（包括学校各类资金既往支持项目） 。 
（五）违背科研伦理、生物安全、信息安全、人类遗传
资源管理、干细胞临床研究、新一代人工智能等管理要求和
</ctx>
------
<hs>
Human: 项目实施过程中，有没有可能被终止项目
AI: 根据提供的上下文，项目实施过程中有可能被终止项目。具体来说，如果项目负责人在项目结题前离职，或者项目负责人出国半年以上未向科学研究院报备或出国逾期不归导致研究工作无法开展，或者未通过年度评估，或者未通过结题验收，或者因其他不可抗力致使研究工作不能正常进行，这些情况一经核实，项目将会被终止。
</hs>
------
项目被终止有什么后果
Answer:


> Finished chain.

> Finished chain.


In [19]:
print(result['result'])

根据提供的上下文，项目被终止可能会对研究院的研究工作产生以下后果：

1. 研究工作无法继续开展：由于项目负责人离职、出国未报备或逾期不归等原因导致项目无法正常开展，从而影响研究院的研究工作。

2. 经费收归：项目被终止后，剩余经费可能会被收回，不再用于该项目。

3. 学术记录受损：如果项目负责人存在违法违纪、违反师德、学术不端等行为，或者项目未通过年度评估、结题验收等，这些情况可能会被记录在研究院的学术记录中，对研究院的声誉产生负面影响。

4. 研究计划变更：对于一些需要变更研究计划的情况，如项目延期、终止等，需要经过研究院审批后方可进行，否则可能会导致研究计划无法继续实施。


In [20]:
result

{'query': '项目被终止有什么后果',
 'result': '根据提供的上下文，项目被终止可能会对研究院的研究工作产生以下后果：\n\n1. 研究工作无法继续开展：由于项目负责人离职、出国未报备或逾期不归等原因导致项目无法正常开展，从而影响研究院的研究工作。\n\n2. 经费收归：项目被终止后，剩余经费可能会被收回，不再用于该项目。\n\n3. 学术记录受损：如果项目负责人存在违法违纪、违反师德、学术不端等行为，或者项目未通过年度评估、结题验收等，这些情况可能会被记录在研究院的学术记录中，对研究院的声誉产生负面影响。\n\n4. 研究计划变更：对于一些需要变更研究计划的情况，如项目延期、终止等，需要经过研究院审批后方可进行，否则可能会导致研究计划无法继续实施。',
 'source_documents': [Document(page_content='科学研究院可视情况撤销、 终止项目， 停拨、 收回剩余经费：  \n（一）存在违法违纪、违反师德、学术不端等行为 。', metadata={'source': 'data/附件2 中山大学关于印发《中山大学高校基本科研业务费管理实施细则》通知.pdf', 'page': 3}),
  Document(page_content='（二）项目负责人在项目结题前离职 。 \n（三）项目负责人出国半年以上未向科学研究院报备\n或出国逾期不归导致研究工作无法开展 。 \n（四）未通过年度评估 。 \n（五）未通过结题验收 。 \n（六）因其他不可抗力致使研究工作不能正常进行。  \n第十三条  项目负责人应在项目完成时间前 1个月内提', metadata={'source': 'data/附件2 中山大学关于印发《中山大学高校基本科研业务费管理实施细则》通知.pdf', 'page': 4}),
  Document(page_content='计划项目研究方案变更需经二级单位临床研究中心及伦理\n委员会审核通过后报科学研究院审批。  \n各类项目实施过程中，涉及团队核心成员变更、项目\n延期、项目终止的，需经二级单位审核通 过后报科学研究\n院审批。  \n第十二条  项目实施中， 发生下列情况之一， 一经核实，', metadata={'source': 'data/附件2 中山大学关于印发《中山大学高校基